In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_1"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold  Temp_Cold  Temp_Mild  Temp_Warm  Temp_Hot  \
0     148.828353               0          0          1          0         0   
1     159.793757               0          0          1          0         0   
2     111.885594               0          0          0          1         0   
3     168.864941               0          0          1          0         0   
4     171.280754               0          0          1          0         0   
...          ...             ...        ...        ...        ...       ...   
1814  198.225523               0          0          0          0         1   
1815  214.470393               0          0          0          0         1   
1816  129.801991               0          0          0          1         0   
1817  148.541340               0          0          0          0         1   
1818  123.075682               0          0          0          0         1   

      Cloud_Clear  Cloud_Partly_Cloudy  Cloud_Cloud

In [28]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                    1.000000
Ostertag                  0.343797
is_holiday                0.261852
Samstag                   0.261361
Sommer                    0.244808
Werktag                   0.188045
Monat_8                   0.174328
Silvester                 0.144596
Monat_7                   0.136877
Donnerstag                0.136544
Temp_Mild                 0.119302
Montag                    0.110951
wetter_sehr_schön         0.103739
Number_of_ships           0.103427
Number_of_ships_scaled    0.103427
Temp_Warm                 0.097649
zwischen_den_jahren       0.090339
Cloud_ok                  0.075294
Temp_Hot                  0.072790
Ship                      0.072114
Monat_6                   0.068487
VPI                       0.060733
Freitag                   0.056900
Heimspiel                 0.056840
KielerWoche               0.054958
Cloud_Partly_Cloudy       0.053832
Cloud_Clear               0.034175
Wind_Strong               0.022024
Weather_Moderate    

In [29]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


Sonntag                   0.176498
Werktag                   0.090656
VPI                       0.075196
Sommer                    0.043375
Temp_Cold                 0.042753
Winter                    0.042234
is_holiday                0.035391
Frühling                  0.025158
Ostertag                  0.023765
Samstag                   0.023522
Cloud_Cloudy              0.020502
Cloud_ok                  0.020502
Monat_8                   0.018639
Monat_4                   0.017499
Temp_Hot                  0.017245
Monat_11                  0.016713
Temp_Mild                 0.016653
Montag                    0.016499
Cloud_Clear               0.015799
Freitag                   0.015384
Monat_7                   0.015252
Weather_Moderate          0.015035
Dienstag                  0.014640
Mittwoch                  0.014132
Weihnachtsmarkt           0.013634
wetter_sehr_schön         0.011700
Monat_1                   0.010893
Feiertag                  0.010256
Number_of_ships_scal

In [30]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Cloud_Cloudy', 'Wind_Light', 'Wind_Moderate', 'Wind_Strong', 'Number_of_ships', 'Ship', 'Weihnachtsmarkt', 'Monat_12', 'Number_of_ships_scaled', 'Cloud_ok', 'Wind_ok']


In [31]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature           VIF
0           Temp_Very_Cold  1.169477e+08
1                Temp_Cold           inf
2                Temp_Mild           inf
3                Temp_Warm           inf
4                 Temp_Hot  1.500577e+07
5              Cloud_Clear  8.145413e+05
6      Cloud_Partly_Cloudy  1.169889e+06
7             Cloud_Cloudy  1.091062e+05
8               Wind_Light  3.870857e+04
9            Wind_Moderate  1.168463e+05
10             Wind_Strong  2.113227e+06
11            Weather_Good  1.182944e+07
12    Weather_Light_Issues  8.581310e+04
13        Weather_Moderate  3.167107e+05
14          Weather_Severe  1.810797e+07
15             KielerWoche  1.453789e+00
16                  Montag  1.906687e+06
17                Dienstag  2.154440e+06
18                Mittwoch  5.825325e+05
19              Donnerstag  1.072312e+05
20                 Freitag  6.563315e+05
21                 Samstag  1.192699e+05
22                 Sonntag  2.503423e+06
23              

In [32]:
# suche miteinander korrelierrende features

In [33]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Cloud_Cloudy', 'Cloud_Clear'), ('Winter', 'Temp_Cold'), ('Monat_6', 'Sommer'), ('Cloud_ok', 'Cloud_Cloudy'), ('Sonntag', 'Umsatz'), ('Wind_Strong', 'Wind_Moderate'), ('Monat_11', 'Herbst'), ('wetter_sehr_schön', 'Temp_Hot'), ('Sommer', 'Temp_Warm'), ('Wind_ok', 'Wind_Strong'), ('Monat_12', 'Weihnachtsmarkt'), ('Monat_4', 'Markt'), ('Cloud_ok', 'Cloud_Clear'), ('Weather_Moderate', 'Weather_Light_Issues'), ('Monat_2', 'Winter'), ('Monat_10', 'Herbst'), ('Winter', 'Temp_Mild'), ('Monat_1', 'Winter'), ('zwischen_den_jahren', 'Silvester'), ('Cloud_Cloudy', 'Cloud_Partly_Cloudy'), ('Wind_Moderate', 'Wind_Light'), ('Monat_4', 'Frühling'), ('Monat_12', 'Winter'), ('Monat_8', 'Sommer'), ('Monat_5', 'Frühling'), ('Monat_6', 'KielerWoche'), ('Monat_9', 'Herbst'), ('Wind_ok', 'Wind_Moderate'), ('Temp_Mild', 'Temp_Cold'), ('Ship', 'Number_of_ships'), ('Weather_Light_Issues', 'Weather_Good'), ('Monat_3', 'Frühling'), ('Werktag', 'Sonntag'), ('Number_of_ships_scale